# Demo to test and show the functionality of the library

In [9]:
# Importing Functions
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TKAgg') # Needed to have figures display properly. 
import flirimageextractor
import Utils as u
import cv2

In [2]:
filename = "C:\\Users\\Francisco\\Documents\\dataset_termal_destino\\Control_Image termal C_paciente 3 (VC)_flir_control_flir_20190907T092339.jpg"
#filename = "D:\\FLIROne\FLIR_20230426_044637_061.jpg"
#filename = "D:\\FLIROne\FLIR_20230426_044706_939.jpg"
flir = flirimageextractor.FlirImageExtractor(exiftool_path="C:\\Windows\\exiftool.exe")
flir.process_image(filename, RGB=True)

In [3]:
therm = flir.get_thermal_np()
rgb_fullres = flir.get_rgb_np()
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(therm)
plt.title('Raw Thermal Image')
plt.subplot(1,2,2)
plt.imshow(rgb_fullres)
plt.title('RGB Full Resolution Image')
plt.show(block='TRUE') 

In [4]:
colormap_image, Imsalida,temp_img_resized,image_copy = u.extract_images(flir)

Image with offsetx=-1, offsety=37, pipx2=479, pipy2=639, real2ir=1.22885632514954


In [5]:
u.save_thermal_csv(temp_img_resized,"temperature_examples.csv",delimiter=";")

# second part of the demo get the perspective transformation between two images

In [6]:
filename2 = "C:\\Users\\Francisco\\Documents\\dataset_termal_destino\\Control_Image termal C_paciente 3 (VC)_flir_control_flir_20190907T092409.jpg"
flir2 = flirimageextractor.FlirImageExtractor(exiftool_path="C:\\Windows\\exiftool.exe")
flir2.process_image(filename2, RGB=True)
therm2 = flir2.get_thermal_np()
rgb_fullres2 = flir2.get_rgb_np()
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(therm2)
plt.title('Raw Thermal Image')
plt.subplot(1,2,2)
plt.imshow(rgb_fullres2)
plt.title('RGB Full Resolution Image')
plt.show(block='TRUE') 
colormap_image2, Imsalida2,temp_img_resized2,image_copy2 = u.extract_images(flir2)
u.save_thermal_csv(temp_img_resized2,"temperature_examples2.csv",delimiter=";")

Image with offsetx=-1, offsety=37, pipx2=479, pipy2=639, real2ir=1.22885632514954


In [11]:
output_image,homography_matrix=u.matchbydescriptors(image_copy,image_copy2)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(output_image)
plt.title('registration between images')
plt.subplot(1,2,2)
plt.imshow(cv2.addWeighted(image_copy, 1 - 0.5, image_copy2, 0.5, 0))
plt.title('Pair of images witout registration')
plt.show(block='TRUE') 